<a href="https://colab.research.google.com/github/dafrie/fin-disclosures-nlp/blob/master/notebooks/ColabLabelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labelling using Colab [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dafrie/fin-disclosures-nlp/blob/master/notebooks/ColabLabelling.ipynb)

- Uses custom widgets for labelling
- Loads and stores the data in a google storage bucket



## Setup / Initialization

In [8]:
# Get the project code
!git clone https://github.com/dafrie/fin-disclosures-nlp.git

# Install the extra spacy model
!python -m spacy download en_core_web_md
# NOTE: Need to restart the runtime afterwards!
exit()

fatal: destination path 'fin-disclosures-nlp' already exists and is not an empty directory.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
/content/fin-disclosures-nlp


In [ ]:
%%bash
echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
sudo apt-get -y -q update
sudo apt-get -y -q install gcsfuse

In [2]:
BUCKET = 'fin-disclosures-nlp' #@param {type:"string"}
assert BUCKET, '*** Must specify an existing GCS bucket name ***'

from google.colab import auth
auth.authenticate_user()

!mkdir -p /content/bucket
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 fin-disclosures-nlp /content/bucket

Using mount point: /content/bucket
Opening GCS connection...
Opening bucket...
Mounting file system...
daemonize.Run: readFromProcess: sub-process: mountWithArgs: mountWithConn: Mount: mount: running fusermount: exit status 1

stderr:
fusermount: mountpoint is not empty
fusermount: if you are sure this is safe, use the 'nonempty' mount option



In [3]:
FIRM_METADATA = "/content/bucket/data/Firm_Metadata.csv" #@param {type:"string"}
DATA_INPUT_PATH = "/content/bucket/data/annual_reports/" #@param {type:"string"}
MASTER_DATA_PATH = "/content/bucket/data/annual_reports/Firm_AnnualReport_DF.csv" #@param {type:"string"}
LABEL_OUTPUT_FN = 'Firm_AnnualReport_Labels_DF.pkl' #@param {type:"string"}

%cd fin-disclosures-nlp/

/content/fin-disclosures-nlp


## Labelling

In [8]:
import importlib
import pandas as pd

import data
from data.custom_widgets import ReportsLabeler

labeler = ReportsLabeler(master_input_path=MASTER_DATA_PATH, label_output_fn=LABEL_OUTPUT_FN)

Output()

Output()

Output()

In [5]:
%ls

config_example.py  DataDescription.md  environment.yml  notebooks/
data/              docker-compose.yml  LICENSE          README.md
